In [ ]:
%pip install tensorflow
%pip install pandas 
%pip install numpy

^C
Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


Press any key to continue . . . 


In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from feature_engineering.tensor_features import develop_features, floating_conv
from path_finder import path_sorter

from sklearn.preprocessing import StandardScaler
from data_handler import LocalToLargeDataLoader


In [4]:
data_loader = LocalToLargeDataLoader(print_progress=True)
parsed_data = data_loader.load_raw_data(path="../../resources")

Retrieving training data...


In [5]:
print(parsed_data.keys())

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'latitude',
       'longitude', 'vesselId', 'portId', 'etaParsed', 'UN_LOCODE', 'ISO',
       'portLongitude', 'portLatitude'],
      dtype='object')


In [6]:
path_dict = path_sorter(parsed_data)

[===================-] 100.0% complete
[====================] 100.0% complete
Concatting dataframes
Number of dataframes: 152370


In [ ]:
features, y, test_features, y_test = data_loader.load_training_data(path_dict)

features=develop_features(features)
test_features=develop_features(test_features)

Sorting into tests and training data
Developing features with the following columns:  Index(['rot', 'heading', 'navstat', 'etaParsed', 'UN_LOCODE', 'ISO',
       'portLongitude', 'portLatitude', 'time_0', 'time_1', 'cog_1', 'sog_1',
       'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2', 'latitude_2',
       'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3', 'longitude_3',
       'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4', 'time_5',
       'cog_5', 'sog_5', 'latitude_5', 'longitude_5', 'time_6', 'cog_6',
       'sog_6', 'latitude_6', 'longitude_6', 'time_7', 'cog_7', 'sog_7',
       'latitude_7', 'longitude_7', 'time_8', 'cog_8', 'sog_8', 'latitude_8',
       'longitude_8', 'time_9', 'cog_9', 'sog_9', 'latitude_9', 'longitude_9'],
      dtype='object')
Categorizing...
Tokenizing...
Normalizing timestamps...
Derivating locations...
Developing features with the following columns:  Index(['rot', 'heading', 'navstat', 'etaParsed', 'UN_LOCODE', 'ISO',
       'port

In [86]:
print(list(features.keys()))
print(list(test_features.keys()))
print(len(list(test_features.keys())))
missing_columns=[col for col in list(features.keys()) if not col in list(test_features.keys())]
if missing_columns:
    test_features[missing_columns]=0
print(y.head())

['rot', 'heading', 'navstat', 'etaParsed', 'UN_LOCODE', 'ISO', 'portLongitude', 'portLatitude', 'time_0', 'time_1', 'cog_1', 'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2', 'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3', 'longitude_3', 'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4', 'time_5', 'cog_5', 'sog_5', 'latitude_5', 'longitude_5', 'time_6', 'cog_6', 'sog_6', 'latitude_6', 'longitude_6', 'time_7', 'cog_7', 'sog_7', 'latitude_7', 'longitude_7', 'time_8', 'cog_8', 'sog_8', 'latitude_8', 'longitude_8', 'time_9', 'cog_9', 'sog_9', 'latitude_9', 'longitude_9']
['rot', 'heading', 'navstat', 'etaParsed', 'UN_LOCODE', 'ISO', 'portLongitude', 'portLatitude', 'time_0', 'time_1', 'cog_1', 'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2', 'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3', 'longitude_3', 'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4', 'time_5', 'cog_5', 'sog_5', 'latitude_5', 'long

In [87]:
features=features.astype(float)
test_features=test_features.astype(float)

X_train = features.to_numpy()
X_test_np =test_features.to_numpy()
y_train = y.to_numpy()
y_test_np = y_test.to_numpy()


In [88]:
print(X_train)
print(X_test_np.shape[1])
print(y)

[[ 0.00000000e+00  3.00000000e+02  5.00000000e+00 ...  0.00000000e+00
  -0.00000000e+00 -7.92393027e-04]
 [-3.00000000e+00  1.14000000e+02  0.00000000e+00 ...  1.36000000e+01
   5.33415334e+00  8.52808528e-01]
 [ 0.00000000e+00  1.86000000e+02  0.00000000e+00 ...  9.00000000e-01
  -2.74365274e-01 -3.51351351e-01]
 ...
 [ 0.00000000e+00  3.33000000e+02  5.00000000e+00 ...  0.00000000e+00
   9.26784060e-04 -0.00000000e+00]
 [ 0.00000000e+00  4.40000000e+01  0.00000000e+00 ...  1.77000000e+01
   5.65873016e+00  9.44742063e+00]
 [ 0.00000000e+00  1.73000000e+02  5.00000000e+00 ...  0.00000000e+00
  -0.00000000e+00  3.17712470e-03]]
54
        latitude_0  longitude_0
39290     35.45642    139.68042
46876     34.47404    138.54770
6999     -26.74072    153.65371
108418    39.24151    -76.52550
51143     51.31398      3.22757
...            ...          ...
119209    55.53064     12.75990
61294     53.56551      8.56709
60261     51.30153      3.23734
54272     52.64461      3.97908
130678   

In [ ]:
model = Sequential()

# Input layer
input_dim = X_train.shape[1]  # Number of features in your dataset

# Add layers
model.add(Dense(64, input_dim=input_dim, activation='relu'))  # First hidden layer
model.add(LSTM(64))
layers=1
dropout=0.1
nodes=input_dim*2


for layer in range(layers):
    model.add(Dense(nodes, activation='relu'))
    model.add(Dropout(dropout))


# Output layer
# If it's a regression task (predicting a continuous variable like time):
model.add(Dense(units=2, activation='linear')) 

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

In [93]:


# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)


In [94]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, 
                    validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
3048/3048 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 3670555754496.0000 - mae: 18109.6426 - val_loss: 3338.8643 - val_mae: 43.3887
Epoch 2/100
3048/3048 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3339.0774 - mae: 43.3951 - val_loss: 3326.1089 - val_mae: 43.2375
Epoch 3/100
3048/3048 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3337.2214 - mae: 43.2564 - val_loss: 3292.3958 - val_mae: 42.8385
Epoch 4/100
1790/3048 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3337.9153 - mae: 43.1124

KeyboardInterrupt: 

In [57]:
loss, mae = model.evaluate(X_test_np, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

953/953 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2622.5747 - mae: 30.9863
Test Loss: 2610.961669921875, Test MAE: 30.82440757751465


In [58]:
predictions = model.predict(X_test_np) 

print(predictions)

953/953 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
[[36.683044 11.225968]
 [36.683044 11.225968]
 [36.683044 11.225968]
 ...
 [36.683044 11.225968]
 [36.683044 11.225968]
 [36.683044 11.225968]]
